In [89]:
from keras.models import Sequential
from keras.layers import Dense,Activation,LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import re

In [90]:
path = './haruto_shura.txt'

In [91]:
bindata = open(path,'rb').read()

In [92]:
text = bindata.decode('shift_jis')

In [93]:
print('Size of text:', len(text))

Size of text: 60070


In [94]:
#charsにテキストに出てくる文字を整列して格納する
#set()では重複したtextで重複している文字を削除している
#例）set([1,2,3,3,2]) = [1,2,3]
chars = sorted(list(set(text)))

In [95]:
print('Total chars:',len(chars))

Total chars: 1684


In [96]:
#numerate(chars)→forループをiがカウントアップするごとにcharの添字も同時に増やしてくれる。
char_indices = dict((c,i) for i,c in enumerate(chars))

In [97]:
indices_char = dict((i,c) for i,c in enumerate(chars))

In [98]:
maxlen = 40
step = 3
sentences = []
next_chars = []

In [99]:
#sentencesにtextをmaxlen文字ずつ格納する
#next_charsに次の文字（一文字）を格納する（sentencesに格納した文字列の次の文字）
for i in range(0,len(text) - maxlen, step):
    sentences.append(text[i:i + maxlen])
    next_chars.append(text[i + maxlen])

In [100]:
#テキストのベクトル化

In [101]:
#Xを初期化[0]→40文字ずつに区切った文章の合計の数
#[1] → 文章の文字列（maxlen)
#[2] → テキストに出てくる「重複している文字を除いた」文字数
#以上のスケールでbool型で初期化
X = np.zeros((len(sentences),maxlen,len(chars)),dtype=np.bool)

In [102]:
y = np.zeros((len(sentences),len(chars)),dtype=np.bool)

In [103]:
for i,sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        #対象の文字列の中で使用されている文字のコードに「1」をたてる
        X[i,t,char_indices[char]] = 1
    #i番目の文字列の予想文字のコードに１を立てる
    y[i,char_indices[next_chars[i]]] = 1

In [104]:
#モデルを定義する

In [25]:
#kerasで使用するNNの座（Sequential)を定義する
model = Sequential()

In [26]:
#LSTM(第一引数:隠れ層(128) 第二引数：LSTMに入力するデータの形の指定（（バッチ数）（step数）（特徴の次元）)
model.add(LSTM(128,input_shape=(maxlen,len(chars))))

In [28]:
#Dence()；隠れ層のノード数を指定する
model.add(Dense(len(chars)))

In [29]:
#出力層に対しての処理
model.add(Activation('softmax'))

In [30]:
#optimizer→【名詞】最適化ルーチン
#RMSprop→勾配法の一種(Adagradの改良版)0.01は学習率
optimizer = RMSprop(lr=0.01)

In [31]:
#モデルの作成（実行は別）categorical_crossentropyは損失関数の一種
model.compile(loss='categorical_crossentropy',optimizer=optimizer)

In [38]:
#二項分布に従った最大の予測値の入っている添字を返す
def sample(preds, temperature = 1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    #二項分布から対象となるものの確率を取り出す
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [39]:
for iteration in range(1,60):
    print()
    print('-' * 50)
    print('繰り返し回数：', iteration)
    #モデルの学習を行う
    #bach_size(一回に処理するデータの数)
    #epochs(何回データを処理するか)
    model.fit(X, y, batch_size=128,epochs=1)
    #モデルの学習をどこから始めるかランダムで決める
    start_index = random.randint(0,len(text)-maxlen-1)
    
    for diversity in [0.2,0.5,1.0,1.2]:
        print()
        print('-----diversity', diversity)

        generated = ''
        #40文字ずつ取得してsentenceに格納
        sentence = text[start_index:start_index + maxlen]
        generated += sentence
        print('----- Seedを生成しました：'+ sentence + '"')
        sys.stdout.write(generated)

        #次の文字を予測して文字を足していく処理
        #40文字から次の400文字を予測する
        for i in range(400):
            x = np.zeros((1,maxlen,len(chars)))
            #文字列の中から一文字ずつ取り出して使用されている文字にフラグを立てる
            for t,char in enumerate(sentence):
                x[0,t,char_indices[char]] = 1.

            #モデルの予想を行う（verbose=進行状況の表示の制御。0:表示なし　1：プログレスバー 2:各試行ごとに一行
            #0番目の予測値
            preds = model.predict(x, verbose=0)[0]
            
            #次の予測値の可能性が最も高い確率のインデックスを戻す
            next_index = sample(preds,diversity)
            #対象の文字コードを取得
            next_char = indices_char[next_index]

            #予測文字を予測文章に足していく
            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()

        print()


--------------------------------------------------
繰り返し回数： 1
Epoch 1/1
20167/20167 [==============================] - 55s 3ms/step - loss: 2.0748

-----diversity 0.2
----- Seedを生成しました：そこでゆつくりとどまるために
本部まででも乗つた方がいい
今日ならわたくしだ"
そこでゆつくりとどまるために
本部まででも乗つた方がいい
今日ならわたくしだつてゐる
　（あんまり　　しつていてゐる
　　　　それからすこかにすからです）
　（あんまり　　　しつぱいにあまり
　　　　　　（そらですか）
　（どうかにといふおふのは
　　　　　　　　　　　　　　　　　　　　　　　　天です）
　　（いちのんのはひかりのかかしたり
　　　　それからすこかにかたちられたらしい）
あんまりあんまりはひかりのかかしればなにからうがらす
　　（それはすこかにあたまでせう）
　（あんまり　　　しつぱいによつけらうない
　　　　それからすこかにすこかです）
　（あんまり　　したつてゐる
　　　（そんなにひかりのきれをなる）
どんなにかしたらうにつたひとは
あたまはいままへでする
　（あんまり　　しつていてゐる
　　　　　それからすこかにかられてゐる）
そんながそしはしたちにそつてゐる
　　（そのそらはそらは
　　　　　　　　

-----diversity 0.5
----- Seedを生成しました：そこでゆつくりとどまるために
本部まででも乗つた方がいい
今日ならわたくしだ"
そこでゆつくりとどまるために
本部まででも乗つた方がいい
今日ならわたくしだつてきまり
［＃地付き］※［＃始め二重括弧、1-2-54］こんなこここはここでです）
　（ちらんとしくろくてゐるのだ
　　　　そしからすこかにかたちられ）
　　　　　（いちどん　はひかりにかぶれてゐる）
すんとがあんまりくらくらくらくるとき
ひとりのアにかやうなあた
　　　　それどこここよあどこどへて
　　　　青いガラスのあかり
　　　　　　　いまです）
　　（そんなられるのです
　　　　こつちのここならなかかしく）
　　　（ましろばらしいつぱいになまた
　　　　　ここしなさい

切らのと骨影ないて底ぐらへ
ぱい月は磨　ルを万へてゐる
　　（の月だかたい終云の」は飛ねふたヶ分）
　（今外ら5風はそ圏そら）
終り二修ハ八気く澱
げてはわれたばさまで二羽と土へか）
んまで台一してりひめをするからが犬の烏馬に
ん（なら明によつた来て乗へある提だ
ほんた式にあまつたべんだしから
、死摩てあるのは今をちられる者時《はい》光ガで《よてよく》のぼとまで
おれもちアガ天c電アはしさまら空アが
ま　　　それで出す日）森いにゐる
木きのりの立（かけでごければなさまです
た第ややくして痛向んまぶ
つたさうに遠げそしてにきらがらひかけ
れてoぶんつまぐさ岩をうて華け）
a分
かれe月夜岩《かうくそ》
う　（
　青　ほんた発一日もやあぎまでんうぶら）
（皺に式星あかくじがめ

--------------------------------------------------
繰り返し回数： 5
Epoch 1/1
20167/20167 [==============================] - 56s 3ms/step - loss: 1.5647

-----diversity 0.2
----- Seedを生成しました：リカサスートラ）
結晶片岩山地では
燃えあがる雲の銅粉
　　　（向ふが燃え"
リカサスートラ）
結晶片岩山地では
燃えあがる雲の銅粉
　　　（向ふが燃えれて
　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　　

KeyboardInterrupt: 